In [1]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

In [5]:
torch.cuda.empty_cache()

In [2]:
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

In [3]:
txt = files[0].open().read(); txt[:75]

'This movie was absolute trash. The director and stars(?)should be banished '

In [5]:
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks, 30))

(#140) ['This','movie','was','absolute','trash','.','The','director','and','stars(?)should','be','banished','from','making','movies','forever','.','The','paper','-','thin','plot','concerns','a','sleazy','director','played','by','the','sleazy'...]


In [6]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

(#145) ['xxbos','xxmaj','this','movie','was','absolute','trash','.','xxmaj','the','director','and','stars(?)should','be','banished','from','making','movies','forever','.','xxmaj','the','paper','-','thin','plot','concerns','a','sleazy','director','played'...]


In [8]:
defaults.text_proc_rules

[<function fastai.text.core.fix_html(x)>,
 <function fastai.text.core.replace_rep(t)>,
 <function fastai.text.core.replace_wrep(t)>,
 <function fastai.text.core.spec_add_spaces(t)>,
 <function fastai.text.core.rm_useless_spaces(t)>,
 <function fastai.text.core.replace_all_caps(t)>,
 <function fastai.text.core.replace_maj(t)>,
 <function fastai.text.core.lowercase(t, add_bos=True, add_eos=False)>]

In [4]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

In [5]:
dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, 
    splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=16, seq_len=80)

In [8]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj i 'll keep this fast and sweet . xxmaj five girls on their way home from a football game decide to take a ' short cut ' that leads them down a deserted forest - ridden road . xxmaj of course nothing but good things happen to them , and they safely arrive at their destination . \n\n xxmaj alright , they do n't . xxmaj soon they 're hunted down by a deranged chick who has some","xxmaj i 'll keep this fast and sweet . xxmaj five girls on their way home from a football game decide to take a ' short cut ' that leads them down a deserted forest - ridden road . xxmaj of course nothing but good things happen to them , and they safely arrive at their destination . \n\n xxmaj alright , they do n't . xxmaj soon they 're hunted down by a deranged chick who has some severe"
1,", the grass is brown ! xxmaj it 's clear they really are xxup not in xxmaj ceylon in this scene or the scene with the giant reclining xxmaj buddha . xxmaj my advice is to skip this one or at least keep a pot of coffee nearby to keep you awake . xxmaj despite its budget , it 's just not a very good or inspired film . \n\n xxmaj by the way , could xxmaj miss xxmaj taylor","the grass is brown ! xxmaj it 's clear they really are xxup not in xxmaj ceylon in this scene or the scene with the giant reclining xxmaj buddha . xxmaj my advice is to skip this one or at least keep a pot of coffee nearby to keep you awake . xxmaj despite its budget , it 's just not a very good or inspired film . \n\n xxmaj by the way , could xxmaj miss xxmaj taylor have"


In [6]:
learn = language_model_learner(
    dls_lm,
    AWD_LSTM,
    drop_mult=0.3,
    metrics=[accuracy, Perplexity()]
).to_fp16()

In [16]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.164886,4.041723,0.292198,56.924355,23:14


In [17]:
learn.save('2epoch')

Path('/home/kilian/.fastai/data/imdb/models/2epoch.pth')

In [7]:
learn = learn.load('2epoch')

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

In [8]:
learn.save_encoder('finetuned_2e')